In [1]:
import pandas as pd
import numpy as np

In [2]:
job_description = pd.read_csv('training_data.csv')
resume = pd.read_csv('Resume.csv')

In [3]:
job_description.head()

,company_name,job_description,position_title,description_length,model_response
0,Google,minimum qualifications\nbachelors degree or eq...,Sales Specialist,2727,"{\n ""Core Responsibilities"": ""Responsible fo..."
1,Apple,description\nas an asc you will be highly infl...,Apple Solutions Consultant,828,"{\n ""Core Responsibilities"": ""as an asc you ..."
2,Netflix,its an amazing time to be joining netflix as w...,Licensing Coordinator - Consumer Products,3205,"{\n ""Core Responsibilities"": ""Help drive bus..."
3,Robert Half,description\n\nweb designers looking to expand...,Web Designer,2489,"{\n ""Core Responsibilities"": ""Designing webs..."
4,TrackFive,at trackfive weve got big goals were on a miss...,Web Developer,3167,"{\n ""Core Responsibilities"": ""Build and layo..."


In [4]:
resume.head()

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [5]:
print(job_description.columns,resume.columns)
print(job_description.shape,resume.shape)

Index(['company_name', 'job_description', 'position_title',
       'description_length', 'model_response'],
      dtype='object') Index(['ID', 'Resume_str', 'Resume_html', 'Category'], dtype='object')
(853, 5) (2484, 4)


## Analysis

In [7]:
resume.Category.value_counts()

Category
INFORMATION-TECHNOLOGY    120
BUSINESS-DEVELOPMENT      120
FINANCE                   118
ADVOCATE                  118
ACCOUNTANT                118
ENGINEERING               118
CHEF                      118
AVIATION                  117
FITNESS                   117
SALES                     116
BANKING                   115
HEALTHCARE                115
CONSULTANT                115
CONSTRUCTION              112
PUBLIC-RELATIONS          111
HR                        110
DESIGNER                  107
ARTS                      103
TEACHER                   102
APPAREL                    97
DIGITAL-MEDIA              96
AGRICULTURE                63
AUTOMOBILE                 36
BPO                        22
Name: count, dtype: int64

# Model Development

In [9]:
# colums we needd from job_description
# -> company_name, job_description, position_title, model_response

In [10]:
job_description['model_response'] = job_description['model_response'].apply(lambda x:x.replace('\n',''))

In [16]:
import ast
def clean_model_response(str_lite):
    obj = ast.literal_eval(str_lite)
    want_values = obj.values()
    a = []
    for value in want_values:
        if value == 'N/A':
            continue  #aviod the below code if value == 'N/A'
        if isinstance(value,list):
            a.extend(str(item) for item in value)
        else:
            a.append(str(value))
    return "".join(a)

job_description['model_response'] = job_description['model_response'].apply(clean_model_response)

In [18]:
# data preparation
job_description.job_description = job_description.job_description.apply(lambda x:x.replace('\n', ' '))
job_description.position_title = job_description.position_title.apply(lambda x:x.replace(" ",''))
job_description.company_name = job_description.company_name.apply(lambda x:x.replace(" ",''))
job_description['jd'] = job_description.job_description + ' ' + job_description.model_response + ' ' + job_description.position_title + ' ' + job_description.company_name
job_description['jd'] = job_description.jd.apply(lambda x:x.lower())

In [20]:
job_description.jd[0]
# correct format for job description

"minimum qualifications bachelors degree or equivalent practical experience years of experience in saas or productivity tools businessexperience managing enterprise accounts with sales cycles preferred qualifications  years of experience building strategic business partnerships with enterprise customersability to work through and with a reseller ecosystem to scale the businessability to plan pitch and execute a territory business strategyability to build relationships and to deliver results in a crossfunctionalmatrixed environmentability to identify crosspromoting and uppromoting opportunities within the existing account baseexcellent account management writtenverbal communication strategic and analyticalthinking skills about the job as a member of the google cloud team you inspire leading companies schools and government agencies to work smarter with google tools like google workspace search and chrome you advocate the innovative power of our products to make organizations more produc

In [22]:
# stemming and stop words

In [24]:
# resume data preparing
resume.head()

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [26]:
def clean_resume_str(a):
    a = a.replace("\n", "")
    a = a.lower()
    a = a.strip()
    words = a.split()  # Splits the string by any whitespace and removes empty strings
    cleaned_text = " ".join(words)  # Joins the words with a single space
    return cleaned_text

# clean_reusme_str does good enough job
resume['res_dis'] = resume.Resume_str + ' ' + resume.Category
resume['res_dis'] = resume['res_dis'].apply(clean_resume_str)

In [28]:
resume.res_dis[300]

"information technology manager / network engineer professional overview a highly skilled and accomplished information technology manager with over 16 years of expertise in planning, implementing and streamlining it systems to ensure maximum customer satisfaction and business revenue. \u200b strong leader with demonstrated ability to work effectively with individuals at all levels and in all functional areas. exceptional communication and project management skills with the ability to successfully manage multiple priorities and assignments. skills platforms: windows 2000, 2003 server and professional, windows xp (pro and home), win 3.11/95/98, dos 6.22, citrix ica, ms exchange 2000 hardware: hp, ibm, dell, and intel based server and desktop platforms, cisco 2500 switches, sonic wall hardware vpns, hp laser jets, toshiba e-310 color copier, toshiba and sony laptops, inter-tel software: ms office, visio, citrix metaframe xp, symantec: pc anywhere, ghost, antivirus, utilities, win fax; mca

## Vectorizaion

In [31]:
singular_str = pd.concat([job_description.jd,resume.res_dis])
singular_str

0       minimum qualifications bachelors degree or equ...
1       description as an asc you will be highly influ...
2       its an amazing time to be joining netflix as w...
3       description  web designers looking to expand y...
4       at trackfive weve got big goals were on a miss...
                              ...                        
2479    rank: sgt/e-5 non- commissioned officer in cha...
2480    government relations, communications and organ...
2481    geek squad agent professional profile it suppo...
2482    program director / office manager summary high...
2483    storekeeper ii professional summary the purpos...
Length: 3337, dtype: object

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer(stop_words = 'english')
vectorizer.fit(singular_str) #creating a vocabulary of the all the words with tfidf score for each

TfidfVectorizer(stop_words='english')

In [34]:
jd_vector = vectorizer.transform(job_description.jd)
jd_vector = jd_vector.toarray()
jd_vector.shape

(853, 53660)

In [35]:
resume_vector = vectorizer.transform(resume.res_dis)
resume_vector = resume_vector.toarray()
resume_vector.shape

(2484, 53660)

In [36]:
similarity = cosine_similarity(jd_vector,resume_vector)
similarity.shape

(853, 2484)

## Filtering Resumes for a job

In [92]:
similarity

array([[0.03572421, 0.04659755, 0.02900491, ..., 0.0162617 , 0.02388459,
        0.03254256],
       [0.03461825, 0.03282698, 0.01942974, ..., 0.01891675, 0.01782917,
        0.02594371],
       [0.03411643, 0.07730451, 0.02838587, ..., 0.01517223, 0.03281399,
        0.01712128],
       ...,
       [0.01506253, 0.03666057, 0.01935337, ..., 0.02277846, 0.04298586,
        0.02279005],
       [0.09207225, 0.04956934, 0.06673858, ..., 0.02900834, 0.06363327,
        0.03900267],
       [0.20133456, 0.05240731, 0.05906527, ..., 0.10179588, 0.09295456,
        0.06753725]])

In [46]:
def rank_resumes_for_job(job_id, top_n=3):
    if job_id not in job_description.index:
        print("Error: Invalid job ID.")
        return

    job_scores = similarity[job_id] # fetches the similarity score for the particular job
    top_resume_indices = job_scores.argsort()[-top_n:][::-1] # argsort -> sorts the values in ascending order and returns indices, 
    #[-top_n] keeps only the last top_n values and [[::-1]] reverses the list
    
    print(f"\n Top {top_n} resumes for: {job_description.loc[job_id, 'position_title']} ")
    
    for i in top_resume_indices:
        score = job_scores[i]
        resume_id = resume.loc[i, 'ID']
        applicant_category = resume.loc[i, 'Category']
        
        print(f"Resume ID: {resume_id}, Category: {applicant_category}, Match Score: {score:.2f}")



In [48]:
rank_resumes_for_job(0)
rank_resumes_for_job(1)
rank_resumes_for_job(2)


--- Top 3 resumes for: SalesSpecialist ---
Resume ID: 85421438, Category: HEALTHCARE, Match Score: 0.21
Resume ID: 15535920, Category: CONSULTANT, Match Score: 0.17
Resume ID: 80503242, Category: ADVOCATE, Match Score: 0.16

--- Top 3 resumes for: AppleSolutionsConsultant ---
Resume ID: 38896303, Category: CONSULTANT, Match Score: 0.13
Resume ID: 21321598, Category: PUBLIC-RELATIONS, Match Score: 0.13
Resume ID: 48533663, Category: CONSULTANT, Match Score: 0.11

--- Top 3 resumes for: LicensingCoordinator-ConsumerProducts ---
Resume ID: 11432686, Category: CHEF, Match Score: 0.18
Resume ID: 23396633, Category: BUSINESS-DEVELOPMENT, Match Score: 0.16
Resume ID: 15592167, Category: ACCOUNTANT, Match Score: 0.13


In [90]:
a = [85421438,15535920,80503242]
resume[resume.ID.isin(a)]
# checking the res_dis column's values says that the selected resume are quite similar to job description 

,ID,Resume_str,Resume_html,Category,res_dis
522,80503242,"MANAGER, DIGITAL MARKETING AND COMMUN...","<div class=""fontsize fontface vmargins hmargin...",ADVOCATE,"manager, digital marketing and communications ..."
764,85421438,PA MEDIA GROUP Summary Be in...,"<div class=""fontsize fontface vmargins hmargin...",HEALTHCARE,pa media group summary be in a position involv...
1220,15535920,BUSINESS CONSULTANT Professiona...,"<div class=""fontsize fontface vmargins hmargin...",CONSULTANT,business consultant professional summary it bu...


In [110]:
import pickle
model_save_path = 'screener.pkl'

# with open(model_save_path, 'wb') as file:
#     pickle.dump(vectorizer,file)

In [114]:
similarity_score_path = 'similarity_score.pkl'
# with open(similarity_score_path, 'wb') as file:
#     pickle.dump(similarity, file)

In [3]:
resume.head()

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [5]:
hr_text = resume[resume.Category == 'HR']['Resume_html'].iloc[0]
hr_text

'<div class="fontsize fontface vmargins hmargins linespacing pagesize" id="document"> <div class="section firstsection" id="SECTION_NAME500375979" style="\n      padding-top:0px;\n    "> <div class="paragraph PARAGRAPH_NAME firstparagraph" id="PARAGRAPH_500375979_1_326506904" style="\n      padding-top:0px;\n    "> <div class="name" itemprop="name"> <span class="field fName" id="500375979FNAM1"> </span> <span> </span> <span class="field" id="500375979LNAM1"> HR ADMINISTRATOR/MARKETING ASSOCIATE\n\nHR ADMINISTRATOR</span> </div> </div> </div> <div class="section" id="SECTION_SUMM500375981" style="\n      padding-top:0px;\n    "> <div class="heading bottomborder"> <div class="sectiontitle" id="SECTNAME_SUMM500375981"> Summary</div> </div> <div class="paragraph firstparagraph" id="PARAGRAPH_500375981_1_326506917" style="\n      padding-top:0px;\n    "> <div class="field singlecolumn" id="500375981FRFM1"> <p align="LEFT"> Dedicated Customer Service Manager with 15+ years of experience in H

In [6]:
with open("res2.html", "w", encoding="utf-8") as file:
    file.write(hr_text)